<a href="https://colab.research.google.com/github/QuasimRizvi/BigData/blob/main/BigData_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

'/content/spark-3.1.1-bin-hadoop2.7'

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -y ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-03-31 01:32:22--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.129.29, 3.225.89.236, 54.226.184.31, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.129.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14746350 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  14.06M  63.7MB/s    in 0.2s    

2021-03-31 01:32:22 (63.7 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [14746350/14746350]

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  

In [ ]:
ls

adc.json                        spark-3.1.1-bin-hadoop2.7/
ngrok-stable-linux-amd64.zip    spark-3.1.1-bin-hadoop2.7.tgz
ngrok-stable-linux-amd64.zip.1  spark-3.1.1-bin-hadoop2.7.tgz.1
sample_data/


In [ ]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import pyspark
conf = pyspark.SparkConf()
conf.set("spark.driver.memory","8G");
import numpy as np
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.sql.functions import round, col

In [ ]:
project_id = 'api3-307502'

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
!gsutil cp gs://ethereum-bigdata/contracts* /tmp/

Copying gs://ethereum-bigdata/contracts000000000000...
Copying gs://ethereum-bigdata/contracts000000000001...
Copying gs://ethereum-bigdata/contracts000000000002...
Copying gs://ethereum-bigdata/contracts000000000003...
\ [4 files][  2.0 GiB/  2.0 GiB]   15.9 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://ethereum-bigdata/contracts000000000004...
Copying gs://ethereum-bigdata/contracts000000000005...
Copying gs://ethereum-bigdata/contracts000000000006...
Copying gs://ethereum-bigdata/contracts000000000007...
Copying gs://ethereum-bigdata/contracts000000000008...
Copying gs://ethereum-bigdata/contracts000000000009...
Copying gs://ethereum-bigdata/contracts000000000010...
Copying gs://ethereum-bigdata/contracts000000000011...
\ [12 f

In [ ]:
spark = SparkSession(sc)

In [ ]:
transactions000000000000_df = spark.read.format('csv').options(header='true').load('/tmp/transactions000000000000.csv')

In [ ]:
transactions000000000000_df.printSchema()

root
 |-- hash: string (nullable = true)
 |-- nonce: string (nullable = true)
 |-- transaction_index: string (nullable = true)
 |-- from_address: string (nullable = true)
 |-- to_address: string (nullable = true)
 |-- value: string (nullable = true)
 |-- gas: string (nullable = true)
 |-- gas_price: string (nullable = true)
 |-- input: string (nullable = true)
 |-- receipt_cumulative_gas_used: string (nullable = true)
 |-- receipt_gas_used: string (nullable = true)
 |-- receipt_contract_address: string (nullable = true)
 |-- receipt_root: string (nullable = true)
 |-- receipt_status: string (nullable = true)
 |-- block_timestamp: string (nullable = true)
 |-- block_number: string (nullable = true)
 |-- block_hash: string (nullable = true)



In [ ]:
contracts_df = spark.read.format('csv').options(header='true').load('/tmp/contracts.csv')

In [ ]:
contracts_df.printSchema()

root
 |-- address: string (nullable = true)
 |-- bytecode: string (nullable = true)
 |-- is_erc20: string (nullable = true)
 |-- is_erc721: string (nullable = true)
 |-- block_timestamp: string (nullable = true)
 |-- block_number: string (nullable = true)
 |-- block_hash: string (nullable = true)



In [ ]:
transactions000000000000_df.show(1)

+--------------------+-----+-----------------+--------------------+--------------------+-----+-----+------------+--------------------+---------------------------+----------------+------------------------+------------+--------------+--------------------+------------+--------------------+
|                hash|nonce|transaction_index|        from_address|          to_address|value|  gas|   gas_price|               input|receipt_cumulative_gas_used|receipt_gas_used|receipt_contract_address|receipt_root|receipt_status|     block_timestamp|block_number|          block_hash|
+--------------------+-----+-----------------+--------------------+--------------------+-----+-----+------------+--------------------+---------------------------+----------------+------------------------+------------+--------------+--------------------+------------+--------------------+
|0x1a5bbbd3fae1bbc...|  207|              153|0x19c8df7808a38ff...|0xdac17f958d2ee52...|    0|60000|132000001459|0xa9059cbb0000000...|  

In [ ]:
contracts = os.listdir('/tmp')

In [ ]:
contracts = ["/tmp/" +x for x in contracts if "contracts00" in x]
contracts

['/tmp/contracts000000000000',
 '/tmp/contracts000000000003',
 '/tmp/contracts000000000002',
 '/tmp/contracts000000000007',
 '/tmp/contracts000000000011',
 '/tmp/contracts000000000008',
 '/tmp/contracts000000000001',
 '/tmp/contracts000000000005',
 '/tmp/contracts000000000006',
 '/tmp/contracts000000000009',
 '/tmp/contracts000000000004',
 '/tmp/contracts000000000010']

In [ ]:
from functools import reduce
dfContracts = reduce(lambda x,y: x.unionAll(y), 
            [spark.read.format('com.databricks.spark.csv')
                       .load(f, header="true", inferSchema="true") 
             for f in contracts])

In [ ]:
dfContracts.printSchema()

root
 |-- address: string (nullable = true)
 |-- bytecode: string (nullable = true)
 |-- is_erc20: boolean (nullable = true)
 |-- is_erc721: boolean (nullable = true)
 |-- block_timestamp: string (nullable = true)
 |-- block_number: integer (nullable = true)
 |-- block_hash: string (nullable = true)



In [ ]:
dfContracts.createOrReplaceTempView("contracts")

In [ ]:
spark.sql("Select COUNT(*) FROM contracts").show()

+--------+
|count(1)|
+--------+
| 9294164|
+--------+



In [ ]:
contracts_df.createOrReplaceTempView("contractsog")

In [ ]:
spark.sql("SELECT COUNT(*) FROM contractsog").show()

+--------+
|count(1)|
+--------+
|10068122|
+--------+



In [44]:
ls

adc.json  sample_data/  spark-3.1.1-bin-hadoop2.7/
